In [1]:
from predict import load_test_results, TestResults
from validate import join_dict_to_table
import netCDF4
from path import Path
from IPython.display import display
from functools import reduce, partial
from scipy.stats.mstats import gmean

In [2]:
TESTS_RESULTS_DIR = "/home/shibbiry/Dropbox/documents/msu/bachelors_thesis_cluster_topology/test_results/"

In [3]:
def dataset_to_big_table(test_results: TestResults) -> pd.DataFrame:
    """
    Returns DataFrame with columns:
        msg_len, node1, node2, ping_mean, ping_median, ping_std_dev
    """
    tables = (
        join_dict_to_table(getattr(test_results, attr)).rename(columns={"value": column})
        for attr, column in [
            ("medians", "ping_median"),
            ("means", "ping_mean"),
            ("std_dev", "ping_std_dev")
        ]
    )
    return reduce(partial(pd.merge, on=["msg_len", "node1", "node2"]), tables)                 

In [4]:
datasets = [load_test_results(dir_) for dir_ in Path(TESTS_RESULTS_DIR).dirs()]
big_tables = [dataset_to_big_table(dataset) for dataset in datasets]
for table in big_tables:
    table["ping_rel_std_dev"] = table["ping_std_dev"] / table["ping_mean"]

ping_rel_std_dev_geometric_means = pd.DataFrame(
    (gmean(table["ping_rel_std_dev"].dropna()) for table in big_tables),
    columns=["geom_mean_of_ping_rel_std_dev"]
) \
    .assign(name=[dataset.name for dataset in datasets])

In [5]:
ping_rel_std_dev_geometric_means

,geom_mean_of_ping_rel_std_dev,name
0,0.314025,2016-11-04-lom2_100_nodes
1,0.364023,2017-04-29__75_nodes_03
2,0.366339,2016-02-10__110_nodes
3,0.290417,2017-02-12__118_nodes
4,0.369574,2017-04-29__25_nodes_01
5,0.366339,2017-02-10__110_nodes
